In [9]:
from scipy.integrate import odeint
import numpy as np
from numpy import cos, pi
import matplotlib.pyplot as plt
import time
from pertussis import *
from pertussis.model import vaccine_model
from pertussis.params.hetro_model import collect_state0
from pertussis.charts import draw_model, plot_stoch_vars
import pymc as pm
from time import clock
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
unpack(np.ones(20),5,5,[5,5])

M
0
INT
INT
INT


TypeError: unsupported operand type(s) for +=: 'int' and 'list'

In [3]:
# State 0
state_0 = collect_state0()
print(state_0)
state_0 = pack(state_0)
print(state_0)
# time.sleep(0.01)  # makes prints clearer

# # Initial Parameters
# t_start = 1948
# t_end = 2024
# step = 365
# t_end = expand_time(t_end, step=step)
# t_range = np.arange(t_start, t_end + 0, 1)

(array([ 0.01333333,  0.01333333,  0.01333333,  0.01333333,  0.01333333,
        0.01333333,  0.01333333,  0.01333333,  0.01333333,  0.01333333,
        0.01333333,  0.01333333,  0.01333333,  0.01333333,  0.01333333]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

NameError: name 'pack' is not defined